Natural Language Processing - Homework 1 

Pham Lan Phuong - 210120

In [69]:
import re
import nltk
from nltk.util import ngrams
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/lanphgphm/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
with open('testing_data.txt', 'r') as f:
    test = f.read()

with open('training_data.txt', 'r') as f:
    train = f.read()

# Question 1
Find all sentences that contain “to be” verbs (i.e. “is”, “are”, ...) in the training data
file.

In [18]:
sentences = nltk.tokenize.sent_tokenize(train)
be_sentences = [] 
for sentence in sentences:
    # use re.search() because we care if the sentence contains 'be' or not
    contains_be = re.search(r'\b(is|are|am|be|being|been|was|were)\b', sentence)
    if contains_be:
        be_sentences.append(sentence)

be_sentences

['As part of the major, students will be equipped with the foundational knowledge in Computer Science and relevant disciplines.',
 'They will be exposed to essential areas of the CS discipline including theory, systems, and applications.',
 'They will learn about the underlying mathematical ideas that are critical for computation, establish proficiency in the process of designing systems and applications, gain experience in collecting and analyzing data using modern technologies, and begin to develop an understanding for the role of users in the design of systems and applications.',
 'The Computer Science major at Fulbright is designed to prepare students for work in industry or continue their lifelong learning as well as potential graduate-level studies.',
 'All students are first required to take the core courses in Liberal Arts and Science.',
 'In addition to the two courses in “Global Humanities and Social Change”, and “Modern Vietnamese Culture and Society”, they will be exposed t

# Question 2
Build a unigram model and a bigram model (both are with add-one smoothing) from the training data file. Then calculate and compare the perplexity score of these two models
on the testing data file.

## Unigram

### Not using add-one smoothing

In [48]:
def add_one_smoothing(unigram_count, vocab_size):
    for word in unigram_count: 
        unigram_count[word] += 1 
        vocab_size += 1
    return unigram_count, vocab_size

In [51]:
# create a dictionary counting word frequency of all words in corpus 
# create copy of dictionary and add-one smoothing 
train_words = nltk.tokenize.word_tokenize(train)

unigram_count = {}
for word in train_words: 
    if word in unigram_count.keys(): 
        unigram_count[word] += 1
    else: 
        unigram_count[word] = 1
vocab_size = len(unigram_count)


In [56]:
# vocab_size is the number of objects in unigram_count 
unigram_probability = {}

for word in unigram_count.keys():
    unigram_probability[word] = unigram_count[word] / vocab_size

In [65]:
# compute perplexity of the entire test corpus 
# using unigram model without smoothing
def unigram_ppl(test, unigram_probability):
    test_words = nltk.tokenize.word_tokenize(test)
    test_ppl = 1 

    for word in test_words: 
        if word in unigram_probability.keys():
            test_ppl *= 1/unigram_probability[word]

    test_ppl = test_ppl ** (1/len(test_words))
    return test_ppl


During testing, the model may see a word that it has not seen in training. 
To tackle this, I saw someone suggested creating a separate token for the
unseen word called <UNKNOWN>, and assign P(<UNKNOWN>) = 1 / vocab_size. 
This makes sense, as the larger the vocab size, the less likely that the 
model will see a new word in testing. 

However, chapter 4 of the textbook says that it's okay for now to ignore 
unseen words, so that's what I did for the unigram_ppl() function.

In [63]:
unigram_ppl(test, unigram_probability)

25.81394472305185

### Using add-one smoothing

In [53]:
from copy import deepcopy
unigram_count_copy = deepcopy(unigram_count)
vocab_size_copy = deepcopy(vocab_size)

smoothed_unigram_count, smoothed_vocab_size = add_one_smoothing(unigram_count_copy, vocab_size_copy)

In [58]:
smoothed_unigram_probability = {}

for word in smoothed_unigram_count.keys():
    smoothed_unigram_probability[word] = smoothed_unigram_count[word] / smoothed_vocab_size

In [64]:
# compute perplexity of the entire test corpus 
# using unigram model with add-one smoothing
unigram_ppl(test, smoothed_unigram_probability)

37.57928459350777

### Bigram

### Using add-one smoothing

In [73]:
# create a raw bigram count
bigrams = list(ngrams(sequence=train_words, n=2))
bigram_count = {}

for pair in bigrams: 
    if pair in bigram_count.keys(): 
        bigram_count[pair] += 1
    else: 
        bigram_count[pair] = 1

bigram_size = len(bigram_count)

In [75]:
bigram_count_copy = deepcopy(bigram_count)
bigram_size_copy = deepcopy(bigram_size)

smoothed_bigram_count, smoothed_bigram_size = \
    add_one_smoothing(bigram_count_copy, bigram_size_copy)

In [76]:
smoothed_bigram_probability = {}

for pair in smoothed_bigram_count.keys():
    smoothed_bigram_probability[pair] = smoothed_bigram_count[pair] / smoothed_bigram_size

In [78]:
# compute perplexity for the entire test corpus 
def bigram_ppl(test, smoothed_bigram_probability, smoothed_unigram_probability):
    test_words = nltk.tokenize.word_tokenize(test) 
    test_bigrams = list(ngrams(sequence=test_words, n=2))
    test_ppl = 1

    for pair in test_bigrams: 
        context = pair[0]
        if pair in smoothed_bigram_probability.keys():
            cond_prob = smoothed_bigram_probability[pair] / smoothed_unigram_probability[context]
            test_ppl *= 1/cond_prob
            
    
    test_ppl = test_ppl ** (1/len(test_words))
    return test_ppl

In [79]:
bigram_ppl(test, smoothed_bigram_probability, smoothed_unigram_probability)

2.0643605162383167

In [80]:
bigram_ppl(test, smoothed_bigram_probability, unigram_probability)

2.485475943355908